## The random variable X and Y have the following joint probability density 𝑓𝑋𝑌(𝑥, 𝑦) = {𝑒^−𝑥−𝑦 0 < 𝑥 < ∞,0 < 𝑦 < ∞ , 0 𝑒𝑙𝑠𝑒𝑤ℎ𝑒𝑟𝑒 } 

## What is 𝑃(𝑋 < 𝑌) ?

<img src="solution.png">

In [ ]:
def pairs(numbers,k):
    
    max_cycles= 0
    add_iterations = len(numbers) -1

    while(add_iterations>0):
        max_cycles += add_iterations
        add_iterations -= 1
    
    if (k > max_cycles):
        print("Maximum no of pairs allowed is ", max_cycles)
        return 
    
    print ("we will have", k, "pairs")  
    
    count = 1
   # matrix = []
    for i in range(len(numbers)):
        #matrix.append([])
        for j in range(i+1,len(numbers)):
            print(numbers[i],numbers[j])
            if count == k:
                return
            else:
                count +=1
    
     
    return

In [ ]:
numbers = [1,3,5]
k = 3
pairs(numbers,k)

In [ ]:
def findindices(data):
    contact_dict = {}
    email_dict = {}

    for index, user in enumerate(data):                    
        contact_group = contact_dict.get(user[1], set()) 
        email_group = email_dict.get(user[2], set())     
        contact_group.add (index)
        email_group.add (index)
        contact_group.update (email_group)                  # Share info between the two groups
        email_group.update (contact_group)                  
        
        for member in contact_group:
            contact_dict[data[member][1]] = contact_group
        
        for member in email_group:                          
            email_dict[data[member][2]] = email_group
        


    result = {tuple(x) for x in contact_dict.values()}      # use either contact_dict or email_dict will yield the same result
    result = list(result) 
    print (result)
    return

In [ ]:
data = [
 ("username1","phone_number1", "email1"),
 ("usernameX","phone_number1", "emailX"),
 ("usernameZ","phone_numberZ", "email1Z"),
 ("usernameY","phone_numberY", "emailX"),
 ]

findindices(data)

## Implement the Forward propagation & Backward propagation for a three layers Neural Network. X,W and b can be random. 

In [ ]:
# Functions to help with forward propagation.
def linear_forward(A_prev, W, b):
    Z = np.dot(W, A_prev) + b
    cache = (A_prev, W, b)
    return Z, cache


def linear_activation_forward(A_prev, W, b, activation_fn):
    assert activation_fn == "sigmoid" or activation_fn == "tanh" or \
        activation_fn == "relu"

    if activation_fn == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)

    elif activation_fn == "tanh":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = tanh(Z)

    elif activation_fn == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)

    assert A.shape == (W.shape[0], A_prev.shape[1])

    cache = (linear_cache, activation_cache)
    return A, cache


def L_model_forward(X, parameters, hidden_layers_activation_fn="relu"):
    A = X                           
    caches = []                     
    L = len(parameters) // 2        

    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(
            A_prev, parameters["W" + str(l)], parameters["b" + str(l)],
            activation_fn=hidden_layers_activation_fn)
        caches.append(cache)

    AL, cache = linear_activation_forward(
        A, parameters["W" + str(L)], parameters["b" + str(L)],
        activation_fn="sigmoid")
    caches.append(cache)

    assert AL.shape == (1, X.shape[1])
    return AL, caches

In [ ]:
def sigmoid_gradient(dA, Z):
    A, Z = sigmoid(Z)
    dZ = dA * A * (1 - A)

    return dZ


def tanh_gradient(dA, Z):
    A, Z = tanh(Z)
    dZ = dA * (1 - np.square(A))

    return dZ


def relu_gradient(dA, Z):
    A, Z = relu(Z)
    dZ = np.multiply(dA, np.int64(A > 0))

    return dZ


# define helper functions that will be used in L-model back-prop
def linear_backword(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1 / m) * np.dot(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    assert dA_prev.shape == A_prev.shape
    assert dW.shape == W.shape
    assert db.shape == b.shape

    return dA_prev, dW, db


def linear_activation_backward(dA, cache, activation_fn):
    linear_cache, activation_cache = cache

    if activation_fn == "sigmoid":
        dZ = sigmoid_gradient(dA, activation_cache)
        dA_prev, dW, db = linear_backword(dZ, linear_cache)

    elif activation_fn == "tanh":
        dZ = tanh_gradient(dA, activation_cache)
        dA_prev, dW, db = linear_backword(dZ, linear_cache)

    elif activation_fn == "relu":
        dZ = relu_gradient(dA, activation_cache)
        dA_prev, dW, db = linear_backword(dZ, linear_cache)

    return dA_prev, dW, db


def L_model_backward(AL, y, caches, hidden_layers_activation_fn="relu"):
    y = y.reshape(AL.shape)
    L = len(caches)
    grads = {}

    dAL = np.divide(AL - y, np.multiply(AL, 1 - AL))

    grads["dA" + str(L - 1)], grads["dW" + str(L)], grads[
        "db" + str(L)] = linear_activation_backward(
            dAL, caches[L - 1], "sigmoid")

    for l in range(L - 1, 0, -1):
        current_cache = caches[l - 1]
        grads["dA" + str(l - 1)], grads["dW" + str(l)], grads[
            "db" + str(l)] = linear_activation_backward(
                grads["dA" + str(l)], current_cache,
                hidden_layers_activation_fn)

    return grads

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2

    for l in range(1, L + 1):
        parameters["W" + str(l)] = parameters[
            "W" + str(l)] - learning_rate * grads["dW" + str(l)]
        parameters["b" + str(l)] = parameters[
            "b" + str(l)] - learning_rate * grads["db" + str(l)]
    return parameters